<a href="https://colab.research.google.com/github/SRI-CSL/signal-public/blob/signal-demonstration/colabs/signal_api_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SIGNAL**ing Interest Data

**Description:** Installation, basic APIs, and Dashboards, see [HERE](https://github.com/SRI-CSL/SIGNAL/blob/main/reports/milestone-7/signal-demo.org)

**Copyright 2022 SRI International.**

This project is under the GPL3 License. See the [LICENSE](https://www.gnu.org/licenses/gpl-3.0.en.html) file for the full license text.

## &#128640; Getting Started

Install the `SIGNAL API` client

In [ ]:
!curl https://signal.cta.sri.com/client > client.tgz
!tar xzf client.tgz
!pip install -r signal_api_client/requirements.txt
!pip install -e signal_api_client
!pip install ipympl
%cd /content/signal_api_client   

Download the `funcs` utilities repository.

In [1]:
import os
import sys

In [2]:
!git clone https://github.com/hsanchez/funcs.git &> /dev/null

In [3]:
os.chdir(f'./funcs')
!git fetch

In [7]:
# !git checkout activity_roles_detection_colab &> /dev/null

## &#9776; Dependencies

In [4]:
import time
import warnings

import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import List, Any, Dict, Tuple
from datetime import date, datetime

In [5]:
import funcs as utils

In [6]:
from signal_api import signal

## &#9997; Configuration

In [7]:
warnings.filterwarnings("ignore")

In [8]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [9]:
np.random.seed(0)

In [10]:
pd.options.plotting.backend = "plotly"

In [11]:
from IPython import get_ipython
utils.common.set_default_vars(os.environ, ipython_val=get_ipython())

In [12]:
utils.console.stdout.print(utils.common.is_run_in_colab())

True

## &#128272; Login

In [13]:
signal.login()

username?: ··········
password?: ··········


True

## Plot Functions

In [76]:
def bar_plot(value_series: pd.Series, log_y: bool=True, labels: Dict={}, show_legend: bool=False) -> None:
  fig = value_series.plot.bar(log_y=log_y, labels=labels)
  fig.update_traces(showlegend=show_legend)  
  fig.show()

## &#128722; Data

### &#9759; Tables

In [60]:
def df_groupby(input_df: pd.DataFrame, groupby_column: List, sort_values: bool=True) -> pd.Series:
    if sort_values:
        tmp_data = input_df.groupby(input_df[groupby_column]).size().sort_values()
    else:
        tmp_data = input_df.groupby(input_df[groupby_column]).size()
    
    return tmp_data

In [15]:
def get_record_count(table_name: str) -> int:
    query = f"SELECT COUNT(*) FROM {table_name};"
    df_result = signal.query_dataframe(query)
    result = df_result['count'].iloc[0]
    return result

In [16]:
def get_table_columns(table_name: str) -> List[str]:
    query = f"SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'{table_name}';"
    df_result = signal.query_dataframe(query)
    result_list = df_result.column_name.tolist()
    return result_list

In [17]:
def print_table_info(table_name: str) -> None:
    total_records = get_record_count(table_name=table_name)
    table_columns = get_table_columns(table_name=table_name)

    print(f"- In total, there are {total_records:,} records in the {table_name} table.\n"\
          f"-- Table {table_name} contains {len(table_columns)} columns, namely: {table_columns}.\n")

In [18]:
def get_db_records(query: str) -> pd.DataFrame:
    df_result = signal.query_dataframe(query)

    return df_result

In [19]:
TABLES_QUERY = "SELECT * FROM information_schema.tables WHERE table_type='BASE TABLE';"

In [20]:
df_tables = signal.query_dataframe(TABLES_QUERY)

In [21]:
table_names = df_tables.table_name.unique()

In [22]:
print(f"There are {len(table_names)} tables currently present in the SIGNAL database.")

There are 87 tables currently present in the SIGNAL database.


In [23]:
df_tables.head()

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,signal,public,scraped_projects,BASE TABLE,None,None,None,None,None,YES,NO,None
1,signal,public,scraped_patch_series,BASE TABLE,None,None,None,None,None,YES,NO,None
2,signal,public,diff,BASE TABLE,None,None,None,None,None,YES,NO,None
3,signal,public,thread,BASE TABLE,None,None,None,None,None,YES,NO,None
4,signal,public,git_files,BASE TABLE,None,None,None,None,None,YES,NO,None


In [24]:
table_names

array(['scraped_projects', 'scraped_patch_series', 'diff', 'thread',
       'git_files', 'email', 'git_commit_edges', 'pg_statistic',
       'pg_type', 'git_file_changes', 'scraped_patches',
       'alembic_version', 'api_key', 'pg_foreign_table', 'pg_authid',
       'signal_history', 'pg_statistic_ext_data', 'git_repos',
       'user_identities', 'person', 'scraped_patch_submitters',
       'mailing_list', 'git_sigs', 'pg_user_mapping', 'pg_subscription',
       'pg_attribute', 'pg_proc', 'pg_class', 'pg_attrdef',
       'pg_constraint', 'pg_inherits', 'pg_index', 'pg_operator',
       'pg_opfamily', 'pg_opclass', 'pg_am', 'pg_amop', 'pg_amproc',
       'pg_language', 'pg_largeobject_metadata', 'pg_aggregate',
       'pg_statistic_ext', 'pg_rewrite', 'pg_trigger', 'pg_event_trigger',
       'pg_description', 'pg_cast', 'pg_enum', 'pg_namespace',
       'pg_conversion', 'pg_depend', 'pg_database', 'pg_db_role_setting',
       'pg_tablespace', 'pg_auth_members', 'pg_shdepend',
       'p

In [25]:
print_table_info(table_name='email')
print_table_info(table_name='scraped_projects')
print_table_info(table_name='scraped_patch_series')
print_table_info(table_name='diff')
print_table_info(table_name='thread')
print_table_info(table_name='git_files')
print_table_info(table_name='git_commit_edges')
print_table_info(table_name='git_file_changes')
print_table_info(table_name='scraped_patches')
print_table_info(table_name='git_repos')
print_table_info(table_name='user_identities')
print_table_info(table_name='person')
print_table_info(table_name='scraped_patch_submitters')
print_table_info(table_name='mailing_list')
print_table_info(table_name='git_sigs')
print_table_info(table_name='patchwork_person_id')
print_table_info(table_name='git_commits')
print_table_info(table_name='git_xrefs')

- In total, there are 828,219 records in the email table.
-- Table email contains 15 columns, namely: ['id', 'mailing_list_id', 'author_id', 'timestamp_sent', 'timestamp_recv', 'reply_to_url', 'thread_id', 'persuasion', 'reply_to_message_id', 'subject', 'body', 'url', 'clean_body', 'email_id', 'message_id'].

- In total, there are 93 records in the scraped_projects table.
-- Table scraped_projects contains 2 columns, namely: ['id', 'name'].

- In total, there are 280,722 records in the scraped_patch_series table.
-- Table scraped_patch_series contains 3 columns, namely: ['id', 'patchwork_id', 'name'].

- In total, there are 667,972 records in the diff table.
-- Table diff contains 4 columns, namely: ['id', 'email_id', 'file', 'contents'].

- In total, there are 117,508 records in the thread table.
-- Table thread contains 20 columns, namely: ['id', 'deg_max', 'deg_max_2', 'deg_max_3', 'deg_max_4', 'deg_max_5', 'patch', 'emails', 'users', 'start', 'days', 'depth', 'star_nodes', 'h_index

## &#129504; Plots

### &#9759; person Table

In [26]:
# person table
df_person = get_db_records("SELECT * FROM person;")

In [27]:
df_person.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21345 entries, 0 to 21344
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             21345 non-null  int64 
 1   name           21345 non-null  object
 2   email_address  21345 non-null  object
 3   is_bot         21345 non-null  int64 
 4   cluster_id     0 non-null      object
dtypes: int64(2), object(3)
memory usage: 833.9+ KB


In [28]:
df_person.head()

,id,name,email_address,is_bot,cluster_id
0,1,Joel Fernandes,joelaf@NO-ID-FOUND.mhonarc.org,0,None
1,2,richard clark,richard.xnu.clark@mail.gmail.com,0,None
2,3,Takashi Iwai,tiwai@suse.de,0,None
3,4,Lu Baolu,baolu.lu@linux.intel.com,0,None
4,5,Chris Wilson,chris@build.alporthouse.com,0,None


In [62]:
tmp_person_is_bot = df_groupby(input_df=df_person, groupby_column='is_bot')
tmp_person_is_bot

is_bot
1     3656
0    17689
dtype: int64

In [30]:
tmp_person_is_bot

is_bot
1     3656
0    17689
dtype: int64

In [78]:
bar_plot(value_series=tmp_person_is_bot, log_y=False, labels=dict(is_bot="User Type", value="number of records"))

### &#9759; scraped_patches Table

In [50]:
df_scraped_patches = get_db_records("SELECT * FROM scraped_patches;")

In [51]:
df_scraped_patches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 938137 entries, 0 to 938136
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             938137 non-null  int64  
 1   project_id     938137 non-null  int64  
 2   patch_id       938137 non-null  int64  
 3   mail_id        938137 non-null  object 
 4   name           938137 non-null  object 
 5   series_id      935162 non-null  float64
 6   ack_count      938137 non-null  int64  
 7   review_count   938137 non-null  int64  
 8   tested_count   938137 non-null  int64  
 9   success_count  938137 non-null  int64  
 10  warning_count  938137 non-null  int64  
 11  fail_count     938137 non-null  int64  
 12  date           938137 non-null  object 
 13  submitter_id   938137 non-null  int64  
 14  delegate       125878 non-null  object 
 15  state          938137 non-null  object 
 16  commit_hash    48390 non-null   object 
dtypes: float64(1), int64(10), obj

In [52]:
df_scraped_patches.head()

,id,project_id,patch_id,mail_id,name,series_id,ack_count,review_count,tested_count,success_count,warning_count,fail_count,date,submitter_id,delegate,state,commit_hash
0,3488,59,13000777,20221006220840.275-4-jonathan.derrick@linux.dev,[2/2] md/bitmap: Add chunk-count-based bitmap ...,1446.0,0,0,0,0,0,0,2022-10-06T00:00:00,522,None,Superseded,None
1,3489,56,13034662,20221107155825.1644604-12-pierre.gondois@arm.com,"[v2,11/23] arm64: dts: Update cache properties...",1203.0,0,1,0,0,0,0,2022-11-07T00:00:00,411,geert,New,None
2,3490,56,13034661,20221107155825.1644604-11-pierre.gondois@arm.com,"[v2,10/23] arm64: dts: Update cache properties...",1203.0,0,0,0,0,0,0,2022-11-07T00:00:00,411,geert,New,None
3,3491,57,13004984,20221012114429.2341215-6-danishanwar@ti.com,"[v6,5/5] remoteproc: pru: Configure firmware b...",1447.0,0,0,0,0,0,0,2022-10-12T00:00:00,396,None,Superseded,None
4,3492,57,13004985,20221012114429.2341215-5-danishanwar@ti.com,"[v6,4/5] remoteproc: pru: Add pru_rproc_set_ct...",1447.0,0,0,0,0,0,0,2022-10-12T00:00:00,396,None,Superseded,None


In [53]:
tmp_sp_project_id = df_groupby(input_df=df_scraped_patches, groupby_column='project_id')
tmp_sp_project_id

project_id
63         7
16         8
77        19
29        41
17        56
       ...  
52     39650
87     49635
31     70477
76     95940
58    115437
Length: 93, dtype: int64

In [79]:
bar_plot(value_series=tmp_sp_project_id, labels=dict(project_id="Project ID", value="number of records"))

In [58]:
tmp_sp_state = df_groupby(input_df=df_scraped_patches, groupby_column='state')
tmp_sp_state

state
Needs ACK                48
In Next                 124
Under Review            744
Queued                  941
Rejected               2136
Deferred               4252
RFC                    6300
Awaiting Upstream      7308
Handled Elsewhere      7809
Mainlined             14739
Changes Requested     26867
Not Applicable        49217
Accepted              91480
Superseded           135830
New                  590342
dtype: int64

In [80]:
bar_plot(value_series=tmp_sp_state, labels=dict(state="State", value="number of records"))

In [63]:
tmp_sp_project_state = df_scraped_patches.groupby([df_scraped_patches.project_id, df_scraped_patches.state]).count()
tmp_sp_project_state

id  patch_id  mail_id  name  series_id  \
project_id state                                                        
1          New                253       253      253   253        253   
2          New                186       186      186   186        186   
3          Accepted           348       348      348   348        348   
           Changes Requested    4         4        4     4          4   
           New                 73        73       73    73         73   
...                           ...       ...      ...   ...        ...   
93         Handled Elsewhere   89        89       89    89         89   
           In Next              1         1        1     1          1   
           New                 23        23       23    23         23   
           Rejected             2         2        2     2          2   
           Superseded         296       296      296   296        296   

                              ack_count  review_count  tested_count  \
project_id state                                                      
1          New                      253           253           253   
2          New                      186           186           186   
3          Accepted                 348           348           348   
           Changes Requested          4             4             4   
           New                       73            73            73   
...                                 ...           ...           ...   
93         Handled Elsewhere         89            89            89   
           In Next                    1             1             1   
           New                       23            23            23   
           Rejected                   2             2             2   
           Superseded               296           296           296   

                              success_count  warning_count  fail_count  date  \
project_id state                                                               
1          New                          253            253         253   253   
2          New                          186            186         186   186   
3          Accepted                     348            348         348   348   
           Changes Requested              4              4           4     4   
           New                           73             73          73    73   
...                                     ...            ...         ...   ...   
93         Handled Elsewhere             89             89          89    89   
           In Next                        1              1           1     1   
           New                           23             23          23    23   
           Rejected                       2              2           2     2   
           Superseded                   296            296         296   296   

                              submitter_id  delegate  commit_hash  
project_id state                                                   
1          New                         253         0           95  
2          New                         186         0           86  
3          Accepted                    348         0          163  
           Changes Requested             4         0            0  
           New                          73         0            0  
...                                    ...       ...          ...  
93         Handled Elsewhere            89         0            0  
           In Next                       1         0            0  
           New                          23         0            0  
           Rejected                      2         0            0  
           Superseded                  296         0            0  

[602 rows x 15 columns]

In [81]:
bar_plot(value_series=tmp_sp_project_state.id.unstack(), log_y=True, labels=dict(project_id="Project ID", value="number of records", state="Patch States"), show_legend=True)

### &#9759; threads Table

In [83]:
df_thread = get_db_records("SELECT * FROM thread;")

In [84]:
df_thread.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117508 entries, 0 to 117507
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               117508 non-null  int64  
 1   name             117508 non-null  object 
 2   root_message_id  117508 non-null  object 
 3   url              117508 non-null  object 
 4   subject          117508 non-null  object 
 5   patch            117508 non-null  int64  
 6   emails           117508 non-null  int64  
 7   users            117508 non-null  int64  
 8   start            117508 non-null  int64  
 9   days             117508 non-null  int64  
 10  depth            117508 non-null  int64  
 11  star_nodes       117508 non-null  int64  
 12  h_index          117508 non-null  int64  
 13  deg_gini         117508 non-null  float64
 14  deg_max          117508 non-null  float64
 15  deg_max_2        85094 non-null   float64
 16  deg_max_3        58892 non-null   floa

In [85]:
df_thread.head()

,id,name,root_message_id,url,subject,patch,emails,users,start,days,depth,star_nodes,h_index,deg_gini,deg_max,deg_max_2,deg_max_3,deg_max_4,deg_max_5,category
0,1,20200817155032,20200817143833.737102804@linuxfoundation.org,https://lkml.iu.edu/hypermail/linux/kernel/200...,[PATCH 5.8 001/464] tracepoint: Mark __tracepo...,1,475,9,1597677533,1,3,1,1,1.0,467.0,1.0,1.0,1.0,1.0,comet
1,2,20200817151017,20200817143819.579311991@linuxfoundation.org,https://lkml.iu.edu/hypermail/linux/kernel/200...,[PATCH 5.7 001/393] tracepoint: Mark __tracepo...,1,398,6,1597679578,1,2,1,1,1.0,396.0,1.0,0.0,0.0,0.0,comet
2,3,20200929081604,20200929110010.467764689@linuxfoundation.org,https://lkml.iu.edu/hypermail/linux/kernel/200...,[PATCH 5.4 006/388] drm/amd/display: Do not do...,1,389,3,1601379429,0,1,1,1,1.0,388.0,0.0,0.0,0.0,0.0,comb
3,4,20200917220115,20200918020110.2063155-1-sashal@kernel.org,https://lkml.iu.edu/hypermail/linux/kernel/200...,[PATCH AUTOSEL 5.4 001/330] drm/v3d: don't lea...,1,340,10,1600394475,7,4,2,2,1.0,329.0,2.0,1.0,1.0,1.0,tree
4,5,20200817143209,20200817143755.807583758@linuxfoundation.org,https://lkml.iu.edu/hypermail/linux/kernel/200...,[PATCH 5.4 014/270] arm64: dts: qcom: msm8916:...,1,275,6,1597682723,1,2,1,1,1.0,273.0,1.0,0.0,0.0,0.0,comet


In [86]:
tmp_thread_category = df_groupby(input_df=df_thread, groupby_column='category')
tmp_thread_category

category
waterfall      326
stringy      12549
tree         12657
comet        20977
atom         32414
comb         38585
dtype: int64

In [88]:
bar_plot(value_series=tmp_thread_category, log_y=False, labels=dict(category="Thread Category", value="number of records"))

# TESTS

In [1]:

import plotly.express as px
 
df = px.data.iris()
 
fig = px.bar(df, x="sepal_width", y="sepal_length",
             color="species", barmode="group",
             facet_row="species", facet_col="species_id")
 
fig.show()